In [1]:
import re
import math
import pickle
import random
import pandas as pd

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate

In [31]:
#Data Lists
nums = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,30,40,50,60,70,80,90,100]
teluguN = ['సున్న','ఒకటి','రెండు','మూడు','నాలుగు','అయిదు','ఆరు','ఏడు','ఎనిమిది','తొమ్మిది','పది','పదకొండు','పన్నెండు','పదమూడు','పధ్నాలుగు','పదునయిదు','పదహారు','పదిహేడు','పధ్ధెనిమిది','పందొమ్మిది','ఇరవై','ముప్పై','నలభై','యాభై','అరవై','డెబ్బై','ఎనభై','తొంభై','వంద']

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
teluguL = ['ఎ','బి', 'సి', 'డి', 'ఈ', 'ఎఫ్', 'జి', 'హెచ్', 'అయి', 'జె', 'కె', 'ఎల్', 'ఎం', 'ఎన్', 'ఓ', 'పి', 'క్యు', 'అర్', 'ఎస్', 'టి', 'యు', 'వి', 'డబల్యు', 'ఎక్స్', 'వయి', 'జెడ్']

#Dictionaries
telugu_nums = {nums[i]:teluguN[i] for i in range(len(nums))}

telugu_letters ={letters[i]:teluguL[i] for i in range(len(letters))} 

#Special Lists
sp_nums = ['', 'ఒక','ఇద్దరు','ముగ్గురు','నలుగురు','ఐదుగురు','ఆరుగురు','ఏడుగురు']

class_nums = ['', 'ఒకటవ', 'రెండవ', 'మూడవ', 'నాలుగవ', 'ఐదవ', 'ఆరవ', 'ఏడవ', 'ఎనిమిదవ', 'తొమ్మిదవ', 'పదవ', 'పదకొండవ', 'పన్నెండవ']


# Known Phrases
teluguWord = {"anantapur": "అనంతపూర్", "guntur": "గుంటూర్", "chitoor": "చిత్తూరు", "rural": "రూరల్ ", 
                     "telugu": "తెలుగు", "english": "ఇంగ్లీష్", "kannada":"కన్నడ", "colony":"కాలనీ"}

# print(len(telugu_nums))
# print(len(telugu_letters))

# print(telugu_nums[16])
# print(sp_nums[6])
# print(telugu_letters['a'])

In [4]:
# dfile -> data from db
# pfile -> data from pdf

def getData(dfile,pfile):
    data = pickle.load(open(dfile,'rb'))
    df = pickle.load(open(pfile,'rb'))
    
    one =data['udise_sch_code'].to_list()
    two =df['Code'].to_list()
    two = [int(t) for t in two]
    
    intersection = sorted(set(one).intersection(set(two)))

    '''
    print("No. of codes in (1)DB - ", len(set(one)), " ,(2)PDF - ", len(set(two)))
    print("No. of codes NOT in PDF - ", len(set(one)-set(two)))
    print("No. of codes NOT in DB - ", len(set(two)-set(one)))
    print("No. of codes in BOTH - ", len(intersection))
    print("\n")
    '''
    return data, df, intersection

In [30]:

data = 'kolani'
print(transliterate(data, sanscript.ITRANS, sanscript.TELUGU))

కోలని


In [5]:
# To TELUGU TRANSLITERATION
# Language specific code 

#case 0 (end) - pick ONLY from telugu_nums and NO మంది at the end
#case 1 (normal) - pick from telugu_nums and add మంది at the end or sp_nums 
def numToTelugu(Int, case=1):
    Int = int(Int)
    
    # Flag to denote if picking from sp_nums
    sp = 0
    
    if Int > 100:
        num =str(Int)
        
    elif 0<Int<8 and case==1:
        sp = 1
        num =sp_nums[Int]
        
    elif Int<=20 or Int in [20,30,40,50,60,70,80,90,100]:
        num =telugu_nums[Int]
    
    else:
        ones =Int%10
        tens = int((Int/10))*10
        num =telugu_nums[tens]+' '+telugu_nums[ones]
        
    if case and not sp:
        num+=' మంది'
        
    return num+' '


def charsToTelugu(abbr,f):
    tChars =''
    
    for c in abbr:
        if c.isalpha():
            c =telugu_letters[c]+f
        tChars+=c
            
    return tChars

def preprocess(phrase):
    new = ''
    for c in phrase:
        if not c.isalpha() and c not in ['.','_']:
            c =' '+c+' '
        new+=c
    
    return new


def toTelugu(phrase):
    tPhrase = ""
    
    phrase = phrase.lower()
    phrase = preprocess(phrase)
    
    words = phrase.split()
    
    spl = ['mpp','zpp','mpps','zphs','mpups','mjpapbcwrs','kgbv','aprei','nclp','icse','govt','cbse','dnt','apswrei','ssa','kgbvs']
    
    dot = '.'
    other = ''
    
    for word in words:
        tPhrase+=' '
        
        if word in teluguWord.keys():
            tPhrase+=teluguWord[word]
        
        elif word in spl or len(word)<3:
            tAbbr = charsToTelugu(word, other)
            tPhrase+=tAbbr
        
        elif '.' in word or '_' in word:
            part =""
            for c in word:
                if c=='.':
                    if len(part)==1:
                        tPhrase+= charsToTelugu(part,dot)
                    else:
                        tPhrase+= charsToTelugu(part,other)+'.'
                    part =""
                elif c=='_':
                    tPhrase+=charsToTelugu(part,other)+'_'
                    part =""
                elif c==' ':
                    if len(part)<3:
                        tPhrase+= charsToTelugu(part,other)
                    else:
                        tPhrase+=transliterate(part, sanscript.ITRANS, sanscript.TELUGU)+' '
                    part = ""
                else:
                    part+=c
                    
            if part:
                if len(part)<3 or "_" in word:
                    tPhrase+= charsToTelugu(part,other)
                else:
                    tPhrase+=transliterate(part, sanscript.ITRANS, sanscript.TELUGU)+' '
                    
        else:    
            tWord =transliterate(word, sanscript.ITRANS, sanscript.TELUGU) 
            tPhrase+=tWord+' '
    
#     print(phrase, tPhrase.strip())
#     print(tPhrase.strip())
    
    return tPhrase.strip()+' '


In [33]:
# Language specific code

def getIntro(details):
    code = str(details[3]).strip()
    name = toTelugu(details[4].strip())
    village = toTelugu(details[2].strip())
    district = toTelugu(details[0].strip())
    block = toTelugu(details[1].strip())
    
    print("School Name:", details[4].strip(), name)
    
    introOpts = ["ఈ పాఠశాల "+village+"గ్రామం లో ఉన్నది. ఈ గ్రామం "+district+"జిల్లా లో "+block+"పరిధి నందు ఉన్నది. ",
                      "ఈ పాఠశాల "+village+"గ్రామం లో ఉన్నది. "+district+"జిల్లా నందు "+block+"పరిధి లో ఈ గ్రామం ఉన్నది. ",
                      district+"జిల్లా లో "+block+"పరిధి లో "+village+"గ్రామం నందు ఈ పాఠశాల ఉన్నది. ",
                      "ఈ పాఠశాల "+district+"జిల్లా లో "+block+"పరిధి లో గల "+village+"గ్రామము నందు ఉన్నది. "
                    ]
    
    intro = random.choice(introOpts)
    intro = intro+"ఈ పాఠశాల యొక్క ఏకీకృత జిల్లా సమాచార విద్యా వ్యవస్థ (U-DISE) కోడ్ "+code+". "
    
    return intro

# Returns Student's Type and Count of Students
def getStudentStats(counts):
    stuType = counts[1].lower()
    bcount =numToTelugu(counts[2])
    gcount = numToTelugu(counts[3])
    total = numToTelugu(int(counts[2])+int(counts[3]))
    
    sp=' '
    pick = 1
    
    
    if 'co' in stuType:
        opts1 =["ఇది ఒక కో-ఎడ్యుకేషనల్ పాఠశాల. ",
                    "ఇది ఒక బాల బాలికల పాఠశాల. " 
                   ]
        opts2 = ["ఇక్కడ "+bcount+"బాలురు "+gcount+"బాలికలు, మొత్తం "+total+"విద్యార్థులు చదువుతున్నారు. ",
                     "ఈ పాఠశాల లో మొత్తం "+total+"విద్యార్థులు చదువుతున్నారు. వీరిలో "+bcount+"బాలురు "+gcount+"బాలికలు ఉన్నారు. ",
                     "మొత్తం "+total+"విద్యార్థులలో "+bcount+"బాలురు "+gcount+"బాలికలు ఈ పాఠశాల లో చదువుతున్నారు. "
                    ]
        
    elif 'boy' in stuType:
        opts1 = ["ఇది ఒక బాలుర పాఠశాల. ",
                     "ఈ పాఠశాల లో బాలురకు మాత్రమే ప్రవేశం ఉన్నది. ",
                     "బాలురకు మాత్రమే ఈ పాఠశాల నందు ప్రవేశం ఉన్నది. "
                    ]
        opts2 = ["ఇక్కడ చదువుతున్న బాలుర సాంఖ్య "+numToTelugu(counts[2],0)+". ", # 0 signifies, pick only numbers normal numbers (end case)
                     "ఈ పాఠశాల లో "+bcount+"బాలురు చదువుతున్నారు. ",
                      bcount+"బాలురు ఈ పాఠశాల లో చదువుతున్నారు. "
                    ]
        
    elif 'girl' in stuType:
        opts1 = ["ఇది ఒక బాలికల పాఠశాల. ",
                     "ఈ పాఠశాల లో బాలికలకు మాత్రమే ప్రవేశం ఉన్నది. ",
                     "బాలికలకు మాత్రమే ఇక్కడ ప్రవేశం ఉన్నది. "
                  ]
        opts2 = ["ఇక్కడ చదువుతున్న బాలికల సాంఖ్య "+numToTelugu(counts[3],0)+". ",
                     "ఈ పాఠశాల లో "+gcount+"బాలికలు చదువుతున్నారు. ",
                      gcount+"బాలికలు ఇక్కడ చదువుతున్నారు. "
                    ]
    else:
        pick = 0
        
    if pick:
        line1 = random.choice(opts1)
        line2 = random.choice(opts2)
        studentStats =line1+line2
    else:
        studentStats = ''
        
    return studentStats

#Returns Teacher's Stats and Reference
def getTeacherStats(counts):
    sp = ' '
    mint = int(counts[5])
    fint =int(counts[6])
    mcount = numToTelugu(mint)
    fcount = numToTelugu(fint)
    
    # if only male OR female teachers.
    if mint>0 and fint==0:
        line ="ఈ పాఠశాల లో "+mcount+"ఉపాధ్యాయులు ఉన్నారు, వీరిలో అందరూ  ఉపాధ్యాయులే. "
    elif fint>0 and mint==0:
        line ="ఈ పాఠశాల లో "+fcount+"ఉపాధ్యాయులు ఉన్నారు, వీరిలో అందరూ ఉపాధ్యాయినులే. "
    
    #Ekopadhya school
    elif mint==1 and fint==0:
        line ="ఇది ఒక ఏకోపాధ్యాయ పాఠశాల. ఈ పాఠశాల లో ఒక ఉపాధ్యాయుడు ఉన్నారు. "
    elif fint==1 and mint==0:
        line ="ఇది ఒక ఏకోపాధ్యాయ పాఠశాల. ఈ పాఠశాల లో ఒక ఉపాధ్యాయిని ఉన్నారు. "
    
    else:
        total = numToTelugu(mint+fint)
        opts =["ఈ పాఠశాల లో "+mcount+"ఉపాధ్యాయులు "+fcount+"ఉపాధ్యాయినులే, మొత్తం "+total+"ఉపాధ్యాయులు ఉన్నారు. ",
                   "ఈ పాఠశాల లో మొత్తం "+total+"ఉపాధ్యాయులు ఉన్నారు. వీరిలో "+mcount+"ఉపాధ్యాయులు "+fcount+"ఉపాధ్యాయినులు ఉన్నారు. ",
                   "మొత్తం "+total+"ఉపాధ్యాయులు "+mcount+"ఉపాధ్యాయులు "+fcount+"ఉపాధ్యాయినులు ఈ పాఠశాల లో ఉన్నారు. "
                 ]
        line = random.choice(opts)
            
    return line
    
def getDesc(details):
    name =details[4].strip()
    mgnt = details[6].strip()
    
    line = ''
    if "Pvt.Aided" in mgnt:
        line = "ఇది ఒక ప్రైవేటు ఎయిడెడ్ పాఠశాల. "
    elif "Pvt.Unaided" in mgnt:
        line = "ఇది ఒక ప్రైవేటు పాఠశాల. "
    elif "KGBV" in mgnt:
        line = "ఇది ఒక కస్తూర్బా గాంధీ బాలికా విద్యాలయ (KGBV). "
    
    elif "MPPS" in name:
        line = "ఇది ఒక మండల ప్రజా పరిషత్ ప్రాథమిక పాఠశాల (MPPS). "
    elif "MPPUS" in name:
        line = "ఇది ఒక మండల ప్రజా పరిషత్ ప్రాథమికోన్నత పాఠశాల (MPUPS). "
    elif "ZPPS" in name:
        line = "ఇది ఒక జిల్లా పరిషత్ ప్రాథమిక పాఠశాల (ZPPS). "
    elif "ZPHS" in name:
        line = "ఇది ఒక జిల్లా పరిషత్ ఉన్నత పాఠశాల (ZPHS). "
        
    return line

def getInfo(data, df, Id):
    details = data.loc[data['udise_sch_code']==Id].values.tolist()[0]
    counts = df.loc[df['Code']==str(Id)].values.tolist()[0]
    
    return details, counts

def getClass(lo, hi):
    if lo==1:
        line = "ఇక్కడ విద్యార్థులు "+class_nums[hi] +" తరగతి వరకూ విద్యని అభ్యసించవచ్చు. "
    else:
        line = "ఇందులో "+class_nums[lo]+" నుండి "+class_nums[hi]+" తరగతి వరకు విద్యాబోధన కలదు. "
    
    return line
    
def write(data, df, Id):
    article =""
    # Respective row details from pdf and excel data
    details, counts = getInfo(data, df, Id)
    
    #Sentence for name, village, district and block
    intro = getIntro(details)
    
    #Management
    management =toTelugu(details[6].strip())
    opts =[management+"నిర్వహణ లో ఈ పాఠశాల ఉన్నది. ",
               "ఈ పాఠశాల "+management+"నిర్వహణ లో పని చేస్తుంది. "
             ]
    descLine =getDesc(details)
    oLine =random.choice(opts)+descLine
    
    #Classes
    desc = details[5].strip()
    p = re.compile(r'[A-Z]+(\d+)-(\d+)')
    m =re.match(p, desc)
    lo =int(m.group(1)); hi =int(m.group(2))
    cLine = getClass(lo, hi)
    
    #School Medium
    medium = toTelugu(details[7].strip())
    opts = [medium+"మాధ్యమం లో "+random.choice(["పాఠాలు ", "పాఠములు "])+"బోధిస్తారు. ",
                "ఈ పాఠశాల లో "+medium+"మాధ్యమం లో "+random.choice(["పాఠాలు ", "పాఠములు "])+"బోధిస్తారు. "
              ]
    mLine = random.choice(opts)
    
    #School Type & student's counts
    studentStats = getStudentStats(counts)
    
    #Teachers counts + Reference Line
    teacherStats = getTeacherStats(counts)
    
    #Source
#     source ="ఈ వివరములు 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి. ఇవి నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా ఇవ్వబడినవి. " 
    source = "ఈ వివరములు నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి."
    
    article =intro+oLine+cLine+mLine+studentStats+teacherStats+source
    
#     print(article)
    
    return article
    

<h1>Generating Article</h1>

In [35]:
dfile_head = "DBpckls/"
pfile_head = "PDFpckls/"

dist = "ANANTAPUR"
data, df, Ids = getData(dfile_head+dist, pfile_head+dist)

In [36]:
#DEBUG TO GENERATE ONLY ONE ARTICLE
article=write(data, df, 28222501207)
print(article)
print(len(article.split()))

School Name: MPUPS K K COLONY ఎంపియుపిఎస్ కె కె కాలనీ 
ఈ పాఠశాల కక్కలపల్లే  ( రూరల్  ) గ్రామం లో ఉన్నది. అనంతపూర్ జిల్లా నందు అనంతపూర్ పరిధి లో ఈ గ్రామం ఉన్నది. ఈ పాఠశాల యొక్క ఏకీకృత జిల్లా సమాచార విద్యా వ్యవస్థ (U-DISE) కోడ్ 28222501207. ఎంపిపి_జెడ్పిపి స్చోఓల్స్ నిర్వహణ లో ఈ పాఠశాల ఉన్నది. ఇక్కడ విద్యార్థులు ఎనిమిదవ తరగతి వరకూ విద్యని అభ్యసించవచ్చు. తెలుగు మాధ్యమం లో పాఠాలు బోధిస్తారు. ఇది ఒక కో-ఎడ్యుకేషనల్ పాఠశాల. మొత్తం 300 మంది విద్యార్థులలో 130 మంది బాలురు 170 మంది బాలికలు ఈ పాఠశాల లో చదువుతున్నారు. మొత్తం పదకొండు మంది ఉపాధ్యాయులు నలుగురు ఉపాధ్యాయులు ఏడుగురు ఉపాధ్యాయినులు ఈ పాఠశాల లో ఉన్నారు. ఈ వివరములు నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి.
95


In [148]:
def customArticle(order, data, df, Id):
    article =""
    # Respective row details from pdf and excel data
    details, counts = getInfo(data, df, Id)
    
    #Sentence for name, village, district and block
    intro = getIntro(details)
    
    #Management
    management =toTelugu(details[6].strip())
    opts =[management+"నిర్వహణ లో ఈ పాఠశాల ఉన్నది. ",
               "ఈ పాఠశాల "+management+"నిర్వహణ లో పని చేస్తుంది. "
             ]
    descLine =getDesc(details)
    oLine =random.choice(opts)+descLine
    
    #Classes
    desc = details[5].strip()
    p = re.compile(r'[A-Z]+(\d+)-(\d+)')
    m =re.match(p, desc)
    lo =int(m.group(1)); hi =int(m.group(2))
    cLine = getClass(lo, hi)
    
    #School Medium
    medium = toTelugu(details[7].strip())
    opts = [medium+"మాధ్యమం లో "+random.choice(["పాఠాలు ", "పాఠములు "])+"బోధిస్తారు. ",
                "ఈ పాఠశాల లో "+medium+"మాధ్యమం లో "+random.choice(["పాఠాలు ", "పాఠములు "])+"బోధిస్తారు. "
              ]
    mLine = random.choice(opts)
    
    #School Type & student's counts
    studentStats = getStudentStats(counts)
    
    #Teachers counts + Reference Line
    teacherStats = getTeacherStats(counts)
    
    #Source
    #source ="ఈ వివరములు 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి. ఇవి నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా ఇవ్వబడినవి. " 
    source = "ఈ వివరములు నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి."
   
    # Write in custom order
    article = ""
    article_parts =[intro, oLine, cLine, mLine, studentStats, teacherStats, source]
    
    for n in order:
        article+=article_parts[int(n)-1]
    
    
    return article



In [144]:
details, counts = getInfo(data, df, 28222501207)

In [149]:
article=customArticle("1234567", data, df, 28222501207)
print(article)
print(len(article.split()))

ఈ పాఠశాల కక్కలపల్లే  ( రూరల్  ) గ్రామం లో ఉన్నది. ఈ గ్రామం అనంతపూర్ జిల్లా లో అనంతపూర్ పరిధి నందు ఉన్నది. ఈ పాఠశాల యొక్క ఏకీకృత జిల్లా సమాచార విద్యా వ్యవస్థ (U-DISE) కోడ్ 28222501207. ఎంపిపి_జెడ్పిపి స్చోఓల్స్ నిర్వహణ లో ఈ పాఠశాల ఉన్నది. ఇక్కడ విద్యార్థులు ఎనిమిదవ తరగతి వరకూ విద్యని అభ్యసించవచ్చు. ఈ పాఠశాల లో తేలుగు మాధ్యమం లో పాఠములు బోధిస్తారు. ఇది ఒక కో-ఎడ్యుకేషనల్ పాఠశాల. మొత్తం 300 మంది విద్యార్థులలో 130 మంది బాలురు 170 మంది బాలికలు ఈ పాఠశాల లో చదువుతున్నారు. ఈ పాఠశాల లో నలుగురు ఉపాధ్యాయులు ఏడుగురు ఉపాధ్యాయినులే, మొత్తం పదకొండు మంది ఉపాధ్యాయులు ఉన్నారు. ఈ వివరములు నేషనల్ యూనివర్శిటీ ఆఫ్ ఎడ్యుకేషనల్ ప్లానింగ్ అండ్ అడ్మినిస్ట్రేషన్ (NUEPA) జరిపిన సర్వే ఆధారముగా 2011-12 విద్యా సంవత్సరమునకు సంబంధించినవి.
98


In [37]:
#GENERATING MULTIPLE ARTICLES
for dist in ["ANANTAPUR", "CHITTOOR","GUNTUR"]:
    print(dist)
    data, df, Ids = getData(dfile_head+dist, pfile_head+dist)

    failed = []
    for Id in Ids:
        try:
            article =write(data, df, Id)
    #         print("pass")
        except:
            failed.append(Id)
            print('\t',Id)
        
    print('\t',failed)
    print('\t',len(failed))
    print('\t',len(Ids))

ANANTAPUR
	 []
	 0
	 4423
CHITTOOR
	 28231191818
	 [28231191818]
	 1
	 5606
GUNTUR
	 []
	 0
	 4193


In [91]:
def main():
    dfile_head = "DBpckls/"
    pfile_head = "PDFpckls/"
    
    high=0
    big =''
    for dist in ["ANANTAPUR", "CHITTOOR", "GUNTUR"]:
        data, df, ids = getData(dfile_head+dist, pfile_head+dist)
        for Id in ids:
            atricle =write(data, df, Id)



<h1>XML GEN</h1>

In [17]:
dfile_head = "DBpckls/"
pfile_head = "PDFpckls/"

dist = "ANANTAPUR"
data, df, Ids = getData(dfile_head+dist, pfile_head+dist)

In [18]:
article=write(data, df, 28222501207)